<a name="top"> <h1>1. Statistic Analysis</h1> <a>

<strong>Caixabank Tech Datathon</strong></p>


<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>https://www.linkedin.com/in/vicviloria/</em>)</p>

## 1. Library Import

In [2]:
import requests
import pandas as pd
import json
import requests
import zipfile
import os

## 2. Extracción de datos

### 2.1. Transactions_df

In [3]:
# Cargar el dataset de transacciones directamente desde la URL
url = "https://cdn.nuwe.io/challenges-ds-datasets/hackathon-caixabank-data-24/transactions_data.csv"
transactions_df = pd.read_csv(url)

#Export as CSV the df to transactions.csv.
transactions_df.to_csv("./data/raw/transactions_data.csv", index=False)

# Extraer los `client_ids` únicos
client_ids = transactions_df['client_id'].unique()

# Display client_ids.
print(client_ids)

# Display Transactions_df.

transactions_df.head()

[1556  561 1129 ... 1862 1527   22]


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


### 2.2. Client Data y Card data

In [4]:
# URL del archivo ZIP
url = "https://cdn.nuwe.io/challenges-ds-datasets/hackathon-caixabank-data-24/new_data.zip"
zip_file = "new_data.zip"

# Descargar el archivo ZIP
response = requests.get(url)
with open(zip_file, 'wb') as f:
    f.write(response.content)

# Descomprimir el archivo
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("new_data")  # Carpeta donde se descomprimirán los archivos

# Listar archivos descomprimidos
print(os.listdir("new_data"))

# Cargar los datos de los CSV
card_data = pd.read_csv("new_data/cards_data.csv")
client_data = pd.read_csv("new_data/users_data.csv")

['train_fraud_labels.json', '__MACOSX', 'users_data.csv', 'cards_data.csv']


In [5]:
# Display cards_df.
card_data.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [6]:
# Display cards_df.
client_data.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [7]:
# Rename id by client_id.
client_data.rename(columns={'id': 'client_id'}, inplace=True)

In [8]:
# Rename id by client_id.
client_data.rename(columns={'id': 'client_id'}, inplace=True)

# Rename id by card_data.
card_data.rename(columns={'id': 'card_id'}, inplace=True)

# Drop id from transactions_df and rename card_id to id.
transactions_df.rename(columns={'id': 'transactions_id'}, inplace=True)

# Comprueba si hay algun card_id que no este en cards_df.
transactions_df[~transactions_df['card_id'].isin(card_data['card_id'])]

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors


In [11]:
# Merge transactions_df with card_data and client_data.
transactions_df_merged = transactions_df.merge(card_data, on='card_id', how='inner')
# Drop client_id_y from transactions_df_merged and rename client_id_x to client_id.
transactions_df_merged.drop(columns=['client_id_y'], inplace=True)
transactions_df_merged.rename(columns={'client_id_x': 'client_id'}, inplace=True)
# Merge transactions_df with client_data.
transactions_df_merged = transactions_df_merged.merge(client_data, on='client_id', how='inner')

# Display transactions_df.
transactions_df_merged.head()

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,...,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,...,7,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,...,6,Male,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,...,4,Male,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,...,5,Female,903 Hill Boulevard,41.42,-87.35,$26168,$53350,$128676,685,5
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,...,5,Male,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2


In [12]:
#Set pandas to check all columns.
pd.set_option('display.max_columns', None)

# Display transactions_df_merged.
transactions_df_merged.head()

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.80,-100.76,$23679,$48277,$110153,740,4
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN,Mastercard,Credit,5175842699412235,12/2024,438,YES,1,$9100,09/2005,2015,No,48,67,1971,6,Male,604 Pine Street,40.80,-91.12,$18076,$36853,$112139,834,5
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN,Mastercard,Debit,5874992802287595,05/2020,256,YES,1,$14802,01/2006,2008,No,49,65,1970,4,Male,2379 Forest Lane,33.18,-117.29,$16894,$34449,$36540,686,3
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN,Mastercard,Debit,5346827663529174,10/2024,54,NO,2,$37634,05/2004,2006,No,52,67,1967,5,Female,903 Hill Boulevard,41.42,-87.35,$26168,$53350,$128676,685,5
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN,Visa,Debit,4354185735186651,01/2020,120,YES,1,$19113,07/2009,2014,No,51,69,1968,5,Male,166 River Drive,38.86,-76.60,$33529,$68362,$96182,711,2


In [15]:
transactions_df_merged.to_csv("transactions_data_merged.csv", index=False)

In [16]:
# Check transactions with client_id 1556.
transactions_df_merged[transactions_df_merged['client_id'] == 1556]

,transactions_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
6,7475334,2010-01-01 00:09:00,1556,2972,$77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
12,7475340,2010-01-01 00:26:00,1556,2972,$39.63,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
1511,7477136,2010-01-01 12:05:00,1556,2972,$15.05,Swipe Transaction,32076,Bismarck,ND,58504.0,5813,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
2140,7477894,2010-01-01 14:45:00,1556,2972,$21.38,Online Transaction,16798,ONLINE,NaN,NaN,4121,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13299903,23754426,2019-10-30 12:11:00,1556,4333,$15.88,Chip Transaction,32076,Bismarck,ND,58504.0,5813,NaN,Mastercard,Debit,5225389722526534,09/2023,891,YES,2,$24772,08/2010,2010,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
13300340,23754981,2019-10-30 14:04:00,1556,4333,$5.35,Chip Transaction,20519,New England,ND,58647.0,5942,NaN,Mastercard,Debit,5225389722526534,09/2023,891,YES,2,$24772,08/2010,2010,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
13301395,23756272,2019-10-30 19:18:00,1556,2972,$5.65,Chip Transaction,43632,Bismarck,ND,58504.0,5310,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4
13305435,23761271,2019-10-31 19:19:00,1556,2972,$5.16,Chip Transaction,37972,Bismarck,ND,58504.0,5300,NaN,Mastercard,Debit (Prepaid),5497590243197280,07/2022,306,YES,2,$55,05/2008,2008,No,30,67,1989,7,Female,594 Mountain View Street,46.8,-100.76,$23679,$48277,$110153,740,4


In [108]:
transactions_df_merged.dtypes

transactions_id            int64
date                      object
client_id                  int64
card_id                    int64
amount                    object
use_chip                  object
merchant_id                int64
merchant_city             object
merchant_state            object
zip                      float64
mcc                        int64
errors                    object
card_brand                object
card_type                 object
card_number                int64
expires                   object
cvv                        int64
has_chip                  object
num_cards_issued           int64
credit_limit              object
acct_open_date            object
year_pin_last_changed      int64
card_on_dark_web          object
current_age                int64
retirement_age             int64
birth_year                 int64
birth_month                int64
gender                    object
address                   object
latitude                 float64
longitude 

## 3. Cleaning Formats

In [114]:
# Clean the variables amount, credit_limit, per_capita_income, yearly_income, total_debt removing the $ symbol and converting to float.
transactions_df_merged['amount'] = transactions_df_merged['amount'].str.replace('$', '').astype(float)
transactions_df_merged['credit_limit'] = transactions_df_merged['credit_limit'].str.replace('$', '').astype(float)
transactions_df_merged['per_capita_income'] = transactions_df_merged['per_capita_income'].str.replace('$', '').astype(float)
transactions_df_merged['yearly_income'] = transactions_df_merged['yearly_income'].str.replace('$', '').astype(float)
transactions_df_merged['total_debt'] = transactions_df_merged['total_debt'].str.replace('$', '').astype(float)

In [115]:
# Convert the variables expire and acct_open_date to datetiem format as MM/YYYY.
transactions_df_merged['expires'] = pd.to_datetime(transactions_df_merged['expires'], format='%m/%Y')
transactions_df_merged['acct_open_date'] = pd.to_datetime(transactions_df_merged['acct_open_date'], format='%m/%Y')

## 4. Task 1

### Query 1

The card_id with the latest expiry date and the lowest credit limit amount.

In [116]:
# Consulta directa para encontrar la card_id deseada
result = (
    transactions_df_merged
    .sort_values(by=['expires', 'credit_limit'], ascending=[False, True])  # Ordenar primero por expiry (descendente) y luego por credit_limit (ascendente)

    .head(1)  # Obtener la primera fila, que será la tarjeta con la fecha de caducidad más reciente y el límite de crédito más bajo
)
result[['card_id', 'expires', 'credit_limit']]

,card_id,expires,credit_limit
8020,4490,2024-12-01,46.0


### Query 2

 The client_id that will retire within a year that has the lowest credit score and highest debt.

In [117]:
# Encontrar el client_id que se retirará dentro de un año, con el menor credit_score y mayor total_debt
client_id_query_2 = (
    transactions_df_merged[
        (transactions_df_merged['retirement_age'] <= transactions_df_merged['current_age'] + 1)
    ]
    .sort_values(by=['credit_score', 'total_debt'], ascending=[True, False])
    .iloc[0]['client_id'] if not transactions_df_merged.empty else None
)

print(f"The client_id that will retire within a year, has the lowest credit score and highest debt is: {client_id_query_2}")

The client_id that will retire within a year, has the lowest credit score and highest debt is: 1987


### Query 3

The transaction_id of an Online purchase on a 31st of December with the highest absolute amount (either earnings or expenses).

In [128]:
# Convertir la columna 'date' a formato de fecha
transactions_df_merged['date'] = pd.to_datetime(transactions_df_merged['date'])

# Obtener el transaction_id con el mayor monto absoluto de las transacciones en línea del 31 de diciembre
transaction_id_query_3 = (
    transactions_df_merged
    .query("use_chip == 'Online Transaction' and date.dt.month == 12 and date.dt.day == 31")
    .assign(amount_abs=lambda x: x['amount'].abs())  # Crear una nueva columna para el monto absoluto
    .loc[lambda x: x['amount_abs'] == x['amount_abs'].max(), 'transactions_id']
)

# Obtener el primer resultado (en caso de que haya múltiples)
transaction_id_query_3 = transaction_id_query_3.iloc[0] if not transaction_id_query_3.empty else None

print(f"The transaction_id of the Online purchase on a 31st of December with the highest absolute amount is: {transaction_id_query_3}")

The transaction_id of the Online purchase on a 31st of December with the highest absolute amount is: 10534178


### Query 4

Which client over the age of 40 made the most transactions with a Visa card in February 2016? Please return the client_id, the card_id involved, and the total number of transactions.

In [127]:
result = transactions_df_merged[
    (transactions_df_merged['current_age'] > 40) & 
    (transactions_df_merged['card_brand'] == 'Visa') & 
    (transactions_df_merged['date'].dt.month == 2) & 
    (transactions_df_merged['date'].dt.year == 2016)
].groupby(['client_id', 'card_id']).size().reset_index(name='total_transactions').nlargest(1, 'total_transactions')

client_id = result['client_id'].values[0]
card_id = result['card_id'].values[0]
total_transactions = result['total_transactions'].values[0]

print(f"Client ID: {client_id}, Card ID: {card_id}, Total Transactions: {total_transactions}")

Client ID: 598, Card ID: 2040, Total Transactions: 172


## 5. Task 2